In [ ]:
##Use Random Forest to prepare a model on fraud data 
##treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("C:/Users/Vijay Rathod/Downloads/Fraud_check (1).csv")
df.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [4]:
df.info() ### There is no null values in our data set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [5]:
#Changing the categorical variables into dummies.
fraud = pd.get_dummies(df)
fraud

,Taxable.Income,City.Population,Work.Experience,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES
0,68833,50047,10,1,0,0,0,1,0,1
1,33700,134075,18,0,1,1,0,0,0,1
2,36925,160205,30,1,0,0,1,0,0,1
3,50190,193264,15,0,1,0,0,1,0,1
4,81002,27533,28,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
595,76340,39492,7,0,1,1,0,0,0,1
596,69967,55369,2,0,1,1,0,0,0,1
597,47334,154058,0,1,0,1,0,0,0,1
598,98592,180083,17,0,1,0,1,0,1,0


In [7]:
#Converting the Target variable i.e. Sales into Categorical
fraud ['Category'] = pd.cut(fraud ['Taxable.Income'], 
                        bins=[0,30000, np.inf], 
                        labels=['Low/Mid','High'],
                        include_lowest=True)
fraud 


,Taxable.Income,City.Population,Work.Experience,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES,Category
0,68833,50047,10,1,0,0,0,1,0,1,High
1,33700,134075,18,0,1,1,0,0,0,1,High
2,36925,160205,30,1,0,0,1,0,0,1,High
3,50190,193264,15,0,1,0,0,1,0,1,High
4,81002,27533,28,1,0,0,1,0,1,0,High
...,...,...,...,...,...,...,...,...,...,...,...
595,76340,39492,7,0,1,1,0,0,0,1,High
596,69967,55369,2,0,1,1,0,0,0,1,High
597,47334,154058,0,1,0,1,0,0,0,1,High
598,98592,180083,17,0,1,0,1,0,1,0,High


In [8]:
### Random Forest Classification

In [9]:
array=fraud.values

In [10]:
X=array[:,1:10]
Y=array[:,10]

In [18]:
kfold=KFold(n_splits=10,shuffle=True)
model=RandomForestClassifier(n_estimators=100,max_features=4)
result=cross_val_score(model,X,Y,cv=kfold)
print(result.mean()*100)

74.0


In [19]:
### Incressing CV scores by using Ensemble Techniques

In [20]:
### Bagging

In [21]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
cart=DecisionTreeClassifier()
model=BaggingClassifier(base_estimator=cart,n_estimators=100,random_state=7)
result=cross_val_score(model,X,Y,cv=kfold)
print(result.mean()*100)

73.66666666666667


In [22]:
#### Bossting

In [23]:
from sklearn.ensemble import AdaBoostClassifier
model=AdaBoostClassifier(n_estimators=10,random_state=7)
result=cross_val_score(model,X,Y,cv=kfold)
print(result.mean()*100.0)

78.83333333333333


In [24]:
###Stacking

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [29]:
## iteration 1
# create the sub models

estimators = []
model3 = LogisticRegression(max_iter=500)
estimators.append(('logistic', model3))
model4 = DecisionTreeClassifier()
estimators.append(('cart', model4))
model5 = SVC()
estimators.append(('svm', model5))
model6 = BaggingClassifier(base_estimator=cart, n_estimators=100, random_state=6)
estimators.append(('bagging', model6))
model7 = AdaBoostClassifier(n_estimators=10, random_state=7)
estimators.append(('boosting', model7))

# create the ensemble modelIter

ensemble = VotingClassifier(estimators)
results3 = cross_val_score(ensemble, X, Y, cv=kfold)
print(results3.mean()*100)

79.16666666666667


In [27]:
##iteration 2
# create the sub models

estimators = []
model8 = LogisticRegression(max_iter=500)
estimators.append(('logistic', model8))
model9 = DecisionTreeClassifier()
estimators.append(('cart', model9))
model10 = BaggingClassifier(base_estimator=cart, n_estimators=150, random_state=6)
estimators.append(('bagging', model10))
model11 = AdaBoostClassifier(n_estimators=10, random_state=6)
estimators.append(('boosting', model11))

# create the ensemble model

ensemble = VotingClassifier(estimators)
results4 = cross_val_score(ensemble, X, Y, cv=kfold)
print(results4.mean()*100)

79.0


In [26]:
##iteration 3
# create the sub models

estimators = []
model12 = LogisticRegression(max_iter=500)
estimators.append(('logistic', model12))
model13 = DecisionTreeClassifier()
estimators.append(('cart', model13))
model14 = AdaBoostClassifier(n_estimators=10, random_state=8)
estimators.append(('boosting', model14))

# create the ensemble modSel

ensemble = VotingClassifier(estimators)
results5 = cross_val_score(ensemble, X, Y, cv=kfold)
print(results5.mean()*100)

79.33333333333333


In [30]:
###As we can see the Cross Validation Score is highest for 03rd iteration, we can consider the same model as our Final model.